In [1]:
# TODOs: Operate DB Class
# TODOs: Use SQLite
# TODOs: Implement BM25

from elasticsearch import Elasticsearch

# INIT OBJECT

PORT = "http://localhost:9200"
INDEX_NAME = "wiki_evidence"
errors_before_interrupt = 5
refresh_index_after_insert = False
max_insert_retries = 3
yield_ok = False

wiki_ev = Elasticsearch(
    PORT,
    #http_auth=(es_api_user, es_api_password)
    retry_on_timeout=True,  # should timeout trigger a retry on different node?
)

wiki_ev.elastic_index = INDEX_NAME

wiki_ev

<Elasticsearch(['http://localhost:9200'])>

In [93]:
def search_text(es, query_, k=5):
    results = es.search(
        index = es.elastic_index,
        query = {
            "match": {
                "document.text": query_,
                },
        },
        size=k)

    hits = results["hits"]["hits"]
    doc_ids = [row['_source']["document"]["id"] for row in hits]
    
    # res_ = []
    # for i in hits:
    #     res_.append({
    #         "title": i["_source"]["document"]["title"],
    #         "text": i["_source"]["document"]["text"]
    #     })

    # title = hits[0]["_source"]["document"]["title"]
    # text = hits[0]["_source"]["document"]["text"]

    # return {
    #     "title": title,
    #     "text": text
    # }

    return hits

def search_topic(es, topic, k=5):
    results = es.search(
        index = es.elastic_index,
        body= {
            "size": k,
            "query": {
                "match": {
                    "document.title": topic,
        }}})

    hits = results["hits"]["hits"]
    doc_ids = [row['_source']["document"]["id"] for row in hits]

    title = hits[0]["_source"]["document"]["title"]
    text = hits[0]["_source"]["document"]["text"]

    return {
        "title": title,
        "text": text
    }

query = "government emails"

text = search_text(wiki_ev, query_=query, k=10)

print(text)


[{'_index': 'wiki_evidence', '_id': 'c-vm3YIBFEOmg5hhpYrO', '_score': 18.119564, '_ignored': ['document.text.keyword'], '_source': {'document': {'id': '412023', 'title': 'Rahm Emanuel', 'text': '[Emanuel and his office were found guilty of breaking state law by withholding government emails by transferring them onto his personal phone., In March 2017 Chicago Tribune reported Emanuel released 2,696 emails he had previously withheld., In the emails there were found to be 26 possible violations of lobbying laws.]'}}}, {'_index': 'wiki_evidence', '_id': 's-7r3YIBFEOmg5hhZJex', '_score': 18.119564, '_ignored': ['document.text.keyword'], '_source': {'document': {'id': '412023', 'title': 'Rahm Emanuel', 'text': 'Emanuel and his office were found guilty of breaking state law by withholding government emails by transferring them onto his personal phone. In March 2017 Chicago Tribune reported Emanuel released 2,696 emails he had previously withheld. In the emails there were found to be 26 possib

In [33]:
import json

data = [json.loads(ln) for ln in open("../data/train_cmv_cleaned.jsonl")]
kp = [json.loads(ln) for ln in open("../data/keyphrases.jsonl")]


In [36]:
sample_args = data[0:5]
sample_kp = kp[0:5]

In [125]:
### USING TOPIC SIGNATURES ###
import spacy
import re

# For each query, the top 20 passages ranked by BM25 (Robertson et al., 1995) are retained, per medium. 
# All passages retrieved for the input statement are merged and deduplicated, and they will be ranked as discussed in § 6.3.3.

# TODOs: Sentential Arguments Segments
# TODOs: Top-20 passages
# TODOs: Merge

nlp = spacy.load("en_core_web_sm")

it_ = 2

arg = sample_args[it_]
kp = sample_kp[it_]
kp = kp["keyphrase"][0]


ar_ = arg["arguments"]
ar_ = nlp(ar_)


results = []
count = 0
for sent in ar_.sents:
    count += 1

    kp = str(sent).lower()
    res = search_text(wiki_ev, query_=kp, k=5)

    res_ = "".join(re.sub(r'\[\[(?:[^|\]]*\|)?([^\]]*)]]', "", i["_source"]["document"]["text"]).strip("[]") for i in res)
    results.append(res_)


results

['She cries when they leave., How heartless are we to call somebody like Terri Schiavo a vegetable?, What are we thinking?When we think about this case, we need to think about the message that we are sending to our children and our grandchildren.She cries when they leave. How heartless are we to call somebody like Terri Schiavo a vegetable? What are we thinking?When we think about this case, we need to think about the message that we are sending to our children and our grandchildren.She cries when they leave. How heartless are we to call somebody like Terri Schiavo a vegetable? What are we thinking?When we think about this case, we need to think about the message that we are sending to our children and our grandchildren.How heartless are we to call somebody like Terri Schiavo a vegetable?, What are we thinking?When we think about this case, we need to think about the message that we are sending to our children and our grandchildren., What we do in this Chamber tonight is as important a